# Исследование базы данных приложения для чтения книг

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.Компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

Задача

Проанализировать базу данных — информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг

Цель

Дать рекомендации и сформулировать ценностное предложение для нового продукта.

## Изучение данных

In [1]:
from IPython.display import display
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'], 
                                                         db_config['pwd'], 
                                                         db_config['host'], 
                                                         db_config['port'],
                                                         db_config['db']) 
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# сохраняем коннектор

In [2]:
# смотрим данные для таблицы books

query = '''
SELECT * 
FROM books
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# смотрим данные для таблицы authors

query = '''
SELECT * 
FROM authors
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# смотрим данные для таблицы ratings

query = '''
SELECT * 
FROM ratings
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [5]:
# смотрим данные для таблицы publishers

query = '''
SELECT * 
FROM publishers
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [6]:
# смотрим данные для таблицы reviews

query = '''
SELECT * 
FROM reviews
LIMIT 5
'''
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Запросы

### Найдем сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date >= '2000-01-01' 
'''
pd.io.sql.read_sql(query, con = engine)

,count
0,821


После 01.01.2000 вышла 821 книга

### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [20]:
query = '''
SELECT b.title,
       COUNT(DISTINCT re.review_id) AS count_reviews,
       AVG(r.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS re ON b.book_id = re.book_id
LEFT JOIN ratings AS r ON b.book_id = r.book_id
GROUP BY b.book_id  
ORDER BY count_reviews DESC
'''
pd.io.sql.read_sql(query, con = engine)

,title,count_reviews,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Water for Elephants,6,3.977273
2,The Glass Castle,6,4.206897
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,The Curious Incident of the Dog in the Night-Time,6,4.081081
...,...,...,...
995,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667
996,The Natural Way to Draw,0,3.000000
997,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,Essential Tales and Poems,0,4.000000


Книга с самым большим количеством отзывов - Twilight (Twilight #1)

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [29]:
query = '''
SELECT p.publisher AS publisher, 
       COUNT(b.book_id) AS count_books
FROM publishers AS p
INNER JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages>50
GROUP BY p.publisher_id
ORDER BY count_books DESC
LIMIT(1)

'''
pd.io.sql.read_sql(query, con = engine)

,publisher,count_books
0,Penguin Books,42


ИЗдательство Penguin Books выпустило наибольшее количество книг толще 50 страниц

### Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками

In [46]:
query = '''
SELECT author, 
       AVG(first.avg_rating) AS max_rating
FROM 
    (SELECT b.book_id,
            a.author,
            AVG(r.rating) AS avg_rating
    FROM books AS b
    INNER JOIN ratings AS r ON r.book_id = b.book_id
    INNER JOIN authors AS a ON b.author_id = a.author_id
    GROUP BY a.author,
             b.book_id
    HAVING COUNT(r.rating)>50) AS first
GROUP BY author 
ORDER BY max_rating DESC
LIMIT(1)

'''
pd.io.sql.read_sql(query, con = engine)

,author,max_rating
0,J.K. Rowling/Mary GrandPré,4.283844


У авторов J.K. Rowling/Mary GrandPrе - самая высокая средняя оценка книг (книги с 50 и более оценками)  

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [48]:
query = '''
SELECT 
    AVG(review) as avg_reviews
    FROM 
        (SELECT COUNT(review_id) as review
            FROM reviews
            WHERE username IN
                (SELECT username
                FROM ratings
                GROUP BY
                    username
                HAVING
                    COUNT(ratings) > 50)
                GROUP BY
                    username) AS reviews_first

'''
pd.io.sql.read_sql(query, con = engine)

,avg_reviews
0,24.333333


Чуть больше 24 обзоров на книги от пользователей, которые поставили  больше 50 оценок